# Ensemble Modeling Study with Titanic
- reference : https://www.kaggle.com/yassineghouzam/titanic-top-4-with-ensemble-modeling

## 1. Introduce
## 2. Load and check data
- 2.1 load data
- 2.2 Outlier detection
- 2.3 joining train and test set
- 2.4 check for null and missing values
## 3. Feature analysis
- 3.1 Numerical values
- 3.2 Categorical values
## 4. Filling missing values
- 4.1 Age
## 5. Feature enginerring
- 5.1 Name/Title
- 5.2 Family Size
- 5.3 Cabin
- 5.4 Ticket
## 6. Modeling
- 6.1 Simple modeling
    - 6.1.1 Cross validate models
    - 6.1.2 Hyperparamater tunning for best models
    - 6.1.3 Plot learning curves
    - 6.1.4 Feature importance of the tree based classifiers
- 6.2 Ensemble modeling
    - 6.2.1 Combining models
- 6.3 Prediction
    - 6.3.1 Predict and Submit results

# Summary
- Feature analysis
- Feature engineering
- Modeling

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from collections import Counter

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

sns.set(style = 'white', context = 'notebook', palette = 'deep')


# 2. Load and check data

### 2.1 Load data

In [3]:
train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")
IDtest = test["PassengerId"]

### 2.2 Outlier dectection

- Percentile을 활용하여, 이상치 데이터를 찾습니다.

In [4]:
def detect_outliers(df, n, features):
    outlier_indices = []
    
    for col in features:
        Q1 = np.percentile(df[col], 25)
        Q3 = np.percentile(df[col],75)
        IQR = Q3 - Q1
        
        outlier_step = 1.5*IQR       
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        
        outlier_indices.extend(outlier_list_col)
        
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    
    return multiple_outliers

# detect outliers from Age, Sibsp, Parch and Fare
Outliers_to_drop = detect_outliers(train,2,["Age","SibSp","Parch","Fare"])

In [5]:
train.loc[Outliers_to_drop] 

In [6]:
# Drop Outliers
train = train.drop(Outliers_to_drop, axis = 0).reset_index(drop=True)

In [7]:
## Join train and test datasets in order to obtain the same number of features during categorical conversion
# 범주형 데이터를 수치형 데이터로 변환을 편리하게 하기 위해서 훈련데이터와 테스트데이터를 결합하겠습니다.
train_len = len(train)
dataset =  pd.concat(objs=[train, test], axis=0).reset_index(drop=True)

In [8]:
dataset = dataset.fillna(np.nan)
dataset.isnull().sum()
## Age and Cabin은 결측값의 중요한 부분을 차지합니다.

In [9]:
train.info()
train.isnull().sum()

In [10]:
train.head()

In [11]:
train.dtypes

In [12]:
train.describe()

# 3. Feature analysis

## 3.1 numerical values

In [13]:
# 수치형 데이터와 생존 데이터와의 상관관계를 Heatmap을 활용하여 파악해보겠습니다.
g = sns.heatmap(train[['Survived', 'SibSp', 'Parch', 'Age', 'Fare']].corr(), annot = True, fmt = '.2f', cmap = 'coolwarm')

- Fare Feature제외하고 Survived 상관성이 없다는 것을 의미하지 않는다.
- Feature들을 다시 파악해야할 필요가 있습니다.

In [14]:
#SibSp
g = sns.factorplot(x = 'SibSp', y = 'Survived', data = train, kind= 'bar')
g.despine(left=True)

- SibSp를 파악해봤을 때, 적을수록 생존 가능성이 높음을 파악할 수 있습니다.
- 이것을 활용하여, Category 데이터로써 사용이 가능해집니다.

In [15]:
#Parch
g  = sns.factorplot(x="Parch",y="Survived",data=train,kind="bar", size = 6)
g.despine(left=True)

- 가족 인원수가 1~3명인 경우, 가족이 탑승하지 않거나, 가족이 매우 많은 경우보다 생존률이 더 높음을 알 수 있습니다.

In [16]:
#Age
sns.set(style='white', context='notebook', palette='deep')
g = sns.FacetGrid(train, col = 'Survived')
g = g.map(plt.hist , 'Age')

- 연령 분포는 꼬리 분포, Gaussian 분포임을 알 수 있습니다.
- Age 분포는 생존한 분포와 생존하지 않은 분포가 서로 다름을 알 수 있습니다.
- 나이가 생존과 상관관계가 없더라도, 생존 가능성이 다소 있는 승갱의 연령 범주가 있음을 알 수 있습니다.

In [17]:
g = sns.kdeplot(train['Age'][(train['Survived'] == 0) & (train['Age'].notnull())], color = 'red', shade = True)
g = sns.kdeplot(train['Age'][(train['Survived'] == 1) & (train['Age'].notnull())], color = 'blue', shade = True)
g = g.legend(['Not Survived', 'Survived'])

- 두개의 분포를 겹쳐서 보았을 때, 아기의 경우 생존률이 높음을 알 수 있습니다.

In [18]:
#Fare
dataset['Fare'].isnull().sum()

In [19]:
# 하나의 누락값이 존재하므로, 예측에 엄청난 영향을 끼치지 않는 중앙값을 넣기로 하였습니다.
dataset["Fare"] = dataset["Fare"].fillna(dataset["Fare"].median())

In [20]:
g = sns.displot(dataset['Fare'], color = 'm', kde = True, label = "Skewness : %.2f"%(dataset["Fare"].skew()))
g = plt.legend(loc = 'best')

- log를 씌워서 분포를 scaling 하겠습니다.

In [21]:
dataset["Fare"] = dataset["Fare"].map(lambda i: np.log(i) if i > 0 else 0)

g = sns.displot(dataset["Fare"], color="b", label="Skewness : %.2f"%(dataset["Fare"].skew()), kde = True)
g = plt.legend(loc="best")

## 3.2 Categorical values

In [22]:
#Sex
g = sns.barplot(x = 'Sex', y = 'Survived', data = train)

In [23]:
train[['Sex', 'Survived']].groupby('Sex', as_index = False).mean()

- 여성의 경우 생존률이 남성의 경우보다 매우 높음을 알 수 있습니다.
- 성별은 생존에 있어서 중요한 역할을함 을 알 수 있습니다.
- 논외로, 1997 타이타닉 영화에서 이러한 문구의 문장이 나왔다고 합니다. "Women and children first".

In [24]:
# Pclass
g = sns.catplot(x = 'Pclass', y = 'Survived', data = train, kind = 'bar')
g.despine(left = True)

In [25]:
g = sns.catplot(x = 'Pclass', y = 'Survived', hue = 'Sex', data = train, kind = 'bar')
g.despine(left = True)

- 1등석인 경우의 경우 2,3등석의 경우보다 생존률이 더욱 높음을 알 수 있습니다.

In [26]:
# Embarked
dataset["Embarked"].isnull().sum()
# S가 많이 나오므로, 누락된 데이터를 'S'로 채우도록 하겠습니다.
dataset['Embarked'].value_counts()
dataset['Embarked'].fillna('S', inplace = True)

In [27]:
g = sns.catplot(x = 'Embarked' ,y = 'Survived', kind = 'bar', data = train)
g.despine(left = True)

- 예상해보겠습니다. 1등석에 탑승한 사람이 S,Q에서 탑승한 사람보다 C에서 탑승한 사람이 많음을 파악 할 수 있습니다.

In [28]:
g = sns.catplot(x = 'Pclass', y = 'Survived', data = train, kind = 'bar', hue = 'Embarked')
g.despine(left = True)

In [29]:
g = sns.catplot("Pclass", col="Embarked",  data=train,
                   size=6, kind="count", palette="muted")
g.despine(left=True)
g = g.set_ylabels("Count")

- S,Q에서는 3등석의 인원비율이 현저히 많지만, C의  경우 1등석에서 탑승한 사람의 비율이 매우 많음을 알 수 있다.
- 1등석이 대피과정에서 가장 먼저 탈출했다고 가설을 세울 수 있습니다.

# 4. Filling missing Values

## 4.1 Age
- Age Feature의 경우 256개의 전체 데이터 셋에서 누락됨을 알 수 있습니다.
- 생존 가능성이 더 높은 하위 계층이 존재하므로, 연력 특성을 유지한 채로, 결측값을 대처하는 것이 가장 바람직합니다.


In [30]:
g = sns.catplot(y = 'Age', x = 'Sex', data = dataset, kind = 'box')
g = sns.catplot(y = 'Age', x = 'Sex', data = dataset, kind = 'box', hue = 'Pclass')
g = sns.catplot(y = 'Age', x = 'Parch', data = dataset, kind = 'box')
g = sns.catplot(y = 'Age', x = 'SibSp', data = dataset, kind = 'box')

- 성별 분포는 나이 분포와 동일해보입니다. 그러므로, 성별 정보로는 나이를 예측하기는 힘듭니다.
- 하지만, 1등석의 경우 2,3등 석의 사람들보다 나이가 많음을 알 수 있습니다.
- 게다가 나이가 많을 수록, 부모/자녀, 형제가 많음을 알 수 있습니다.

In [31]:
dataset['Sex'] = dataset['Sex'].map({'male' : 0 , 'female' : 1})
g = sns.heatmap(dataset[["Age","Sex","SibSp","Parch","Pclass"]].corr(),cmap="BrBG",annot=True)

-  나이는 성별과 상관관계가 없음을 알 수 있습니다. 하지만, 다른 Feature들과는 negative 상관관계가 존재합니다.
- Parch의 기능에 따른 연령에서 연령은 부모/자녀의 수에 따라 증가한다.
- SibSp, Parch, Pclass를 활용하여, Age의 누락된 값을 채우도록 하겠습니다.

In [32]:
index_NaN_age = list(dataset['Age'][dataset['Age'].isnull()].index)

for i in index_NaN_age:
    age_med = dataset['Age'].median()
    age_pred = dataset['Age'][((dataset['SibSp'] == dataset.iloc[i]['SibSp'])& (dataset['Parch'] == dataset.iloc[i]['Parch']) & (dataset['Pclass'] == dataset.iloc[i]['Pclass']))].median()
    if not np.isnan(age_pred):
        dataset['Age'].iloc[i] = age_pred
    else:
        dataset['Age'].iloc[i] = age_med

In [33]:
g = sns.catplot(x = 'Survived', y = 'Age', data = train, kind = 'box')
g = sns.catplot(x = 'Survived', y = 'Age', data = train, kind = 'violin')

- 생존한 집단과 생존하지 않은 집단과 연령에서의 중앙값 차이가 거의 없음을 알 수 있습니다.
- 하지만, Violin 그래프로 파악했을때는 나이가 어릴수록 더욱 생존률이 높음을 알 수 있습니다.

# 5. Feature engineering

## 5.1 Name/Title

In [34]:
dataset["Name"].head()

- Name Feature는 승객의 성별을 나타내는 것을 가지고 있습니다.
- 이후로 몇몇 승객들의 고유한 이름이 선호될수 있으므로, 모델에 추가하는 것이 좋을 것이다.

In [35]:
dataset_title = [i.split(',')[1].split('.')[0].strip() for i in dataset['Name']]
dataset['Title'] = pd.Series(dataset_title)
dataset['Title'].head()

In [36]:
g = sns.countplot(x="Title",data=dataset)
g = plt.setp(g.get_xticklabels(), rotation=45)

- 4개로 분류하여 저장하겠습니다.

In [37]:
dataset['Title'].value_counts(ascending = False)

In [38]:
dataset["Title"] = dataset["Title"].replace(['Lady', 'the Countess','Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
dataset["Title"] = dataset["Title"].map({"Master":0, "Miss":1, "Ms" : 1 , "Mme":1, "Mlle":1, "Mrs":1, "Mr":2, "Rare":3})
dataset["Title"] = dataset["Title"].astype(int)

In [39]:
g = sns.countplot(dataset['Title'])
g = g.set_xticklabels(["Master","Miss/Ms/Mme/Mlle/Mrs","Mr","Rare"], rotation = 45)

In [40]:
g = sns.catplot(x = 'Title', y = 'Survived', data = dataset, kind = 'bar')
g = g.set_xticklabels(["Master","Miss/Ms/Mme/Mlle/Mrs","Mr","Rare"])

- 희귀한 성을 가진 사람들이 더욱 생존률이 높다는 것은 특이한 경우입니다.

In [41]:
dataset.drop(labels = ["Name"], axis = 1, inplace = True)

## 5.2 Family Size
- 대가족인 경우 생존하기 힘들다는 것을 알 수 있습니다., 그래서 가족의 규모라는 Feature를 생성하도록 하여, 비교해보겠습니다.

In [42]:
dataset['Fsize'] = dataset['SibSp'] + dataset['Parch'] + 1
g = sns.factorplot(x = 'Fsize', y = 'Survived', data = dataset)

In [43]:
dataset['Single'] = dataset['Fsize'].map(lambda s: 1 if s == 1 else 0)
dataset['SmallF'] = dataset['Fsize'].map(lambda s: 1 if  s == 2  else 0)
dataset['MedF'] = dataset['Fsize'].map(lambda s: 1 if 3 <= s <= 4 else 0)
dataset['LargeF'] = dataset['Fsize'].map(lambda s: 1 if s >= 5 else 0)

g = sns.catplot(x="Single",y="Survived",data=dataset,kind="bar")
g = sns.catplot(x="SmallF",y="Survived",data=dataset,kind="bar")
g = sns.catplot(x="MedF",y="Survived",data=dataset,kind="bar")
g = sns.catplot(x="LargeF",y="Survived",data=dataset,kind="bar")

- 가족의 규모가 작거나, 큰 경우 생존률이 혼자 있는 경우와 대가족인 경우보다 많음을 알 수 있습니다.

In [44]:
dataset = pd.get_dummies(dataset, columns = ["Title"])
dataset = pd.get_dummies(dataset, columns = ["Embarked"], prefix="Em")

In [45]:
dataset.head()

In [46]:
dataset["Cabin"].head()

In [47]:
dataset["Cabin"].describe()

In [48]:
dataset["Cabin"].isnull().sum()
# 객실 번호가 없는 경우 누락된 값을 표시했음 예상해보겠습니다.

In [49]:
dataset["Cabin"][dataset["Cabin"].notnull()].head()

In [50]:
dataset['Cabin'] = pd.Series([i[0] if not pd.isnull(i) else 'X' for i in dataset['Cabin']])

In [51]:
dataset["Cabin"].value_counts()

In [52]:
g = sns.countplot(dataset['Cabin'], order = ['A','B','C','D','E','F','G','T','X'])

In [53]:
g = sns.catplot(x = 'Cabin', y = 'Survived', data = dataset, kind = 'bar', order = ['A','B','C','D','E','F','G','T','X'])

- X석을 제외한 자리는 일반적으로 생존 가능한 기회가 조금더 증가함을 알 수 있습니다.

In [54]:
dataset = pd.get_dummies(dataset, columns = ["Cabin"],prefix="Cabin")

## 5.4 Ticket

In [55]:
dataset['Ticket'].head()

- Ticket의 접두사가 같은 자리를 나타낼수 있음을 알 수 있고, Ticket의 접두사가 유사한 class 또는 Survived 를 나타낼수도 있습니다.
- Ticket의 Column을 Ticket의 접두사로 변경하는 작업을 수행하도록 하겠습니다. 이것이 아마 더 효율적인 정보를 가져올 수 있습니다.

In [56]:
#  Ticket의 접두사를 활용하여, Ticket Feature을 처리합니다. 만약에 접두사가 존재하지 않는 경우 X를 적용합니다.
Ticket = []
for i in list(dataset.Ticket):
    # data가 숫자만으로 이루어지지 않을 때 실행됩니다.
    if not i.isdigit():
        Ticket.append(i.replace(".","").replace("/","").strip().split(' ')[0])
    else:
        Ticket.append('X')
dataset['Ticket'] = Ticket
dataset['Ticket'].head()

In [57]:
dataset = pd.get_dummies(dataset, columns = ["Ticket"], prefix="T")

In [58]:
dataset["Pclass"] = dataset["Pclass"].astype("category")
dataset = pd.get_dummies(dataset, columns = ["Pclass"],prefix="Pc")

In [59]:
# ID는 의미가 없으므로 삭제하도록 하겠습니다.
dataset.drop(labels = ["PassengerId"], axis = 1, inplace = True)

In [60]:
dataset.head()

# 6. MODELING

In [88]:
train = dataset[:train_len]
test = dataset[train_len:]
test.drop(labels = 'Survived', axis = 1, inplace = True)

In [62]:
train["Survived"] = train["Survived"].astype(int)
Y_train = train["Survived"]
X_train = train.drop(labels = "Survived",axis = 1)

## 6.1 모델링
- 10개의 분류 모델을 활용하여 비교할 것이며, 각각에 대하여, 검증절차를 사용하여, 정확도를 파악하겠습니다.
    - SVC
    - Decision Tree
    - AdaBoost
    - Extra Trees
    - Gradient Boosting
    - Neural Network
    - KNN
    - Logistic Regression
    - Linear Discriminant Analysis

In [63]:
kfold = StratifiedKFold(n_splits=10)
# StratifiedKFold를 사용하면 데이터가 한곳으로 몰리는 것을 방지합니다.

In [64]:
random_state = 2
classifiers = []

classifiers.append(SVC(random_state=random_state))
classifiers.append(DecisionTreeClassifier(random_state=random_state))
classifiers.append(AdaBoostClassifier(DecisionTreeClassifier(random_state=random_state),random_state=random_state,learning_rate=0.1))
classifiers.append(RandomForestClassifier(random_state=random_state))
classifiers.append(ExtraTreesClassifier(random_state=random_state))
classifiers.append(GradientBoostingClassifier(random_state=random_state))
classifiers.append(MLPClassifier(random_state=random_state))
classifiers.append(KNeighborsClassifier())
classifiers.append(LinearDiscriminantAnalysis())

cv_results = []
for classifier in classifiers :
    cv_results.append(cross_val_score(classifier, X_train, y = Y_train, scoring = "accuracy", cv = kfold, n_jobs=4))
    
cv_means = []
cv_std = []
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())

cv_res = pd.DataFrame({"CrossValMeans":cv_means,"CrossValerrors": cv_std,"Algorithm":["SVC","DecisionTree","AdaBoost",
"RandomForest","ExtraTrees","GradientBoosting","MultipleLayerPerceptron","KNeighboors","LinearDiscriminantAnalysis"]})

g = sns.barplot("CrossValMeans","Algorithm",data = cv_res, palette="Set3",orient = "h",**{'xerr':cv_std})
g.set_xlabel("Mean Accuracy")
g = g.set_title("Cross validation scores")

- Ensemble 모델로 SVC, AdaBoost, RandomForest, ExtraTrees, GradientBoosting 분류를 사용하겠습니다.

In [65]:
# Adaboost
DTC = DecisionTreeClassifier()

adaDTC = AdaBoostClassifier(DTC, random_state=7)

ada_param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "algorithm" : ["SAMME","SAMME.R"],
              "n_estimators" :[1,2],
              "learning_rate":  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3,1.5]}

gsadaDTC = GridSearchCV(adaDTC,param_grid = ada_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsadaDTC.fit(X_train,Y_train)

ada_best = gsadaDTC.best_estimator_

In [66]:
gsadaDTC.best_score_

In [67]:
#ExtraTrees 
ExtC = ExtraTreesClassifier()


## Search grid for optimal parameters
ex_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsExtC = GridSearchCV(ExtC,param_grid = ex_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsExtC.fit(X_train,Y_train)

ExtC_best = gsExtC.best_estimator_

# Best score
gsExtC.best_score_

In [68]:
RFC = RandomForestClassifier()


## Search grid for optimal parameters
rf_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsRFC.fit(X_train,Y_train)

RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_

In [69]:
GBC = GradientBoostingClassifier()
gb_param_grid = {'loss' : ["deviance"],
              'n_estimators' : [100,200,300],
              'learning_rate': [0.1, 0.05, 0.01],
              'max_depth': [4, 8],
              'min_samples_leaf': [100,150],
              'max_features': [0.3, 0.1] 
              }

gsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsGBC.fit(X_train,Y_train)

GBC_best = gsGBC.best_estimator_

# Best score
gsGBC.best_score_

In [70]:
### SVC classifier
SVMC = SVC(probability=True)
svc_param_grid = {'kernel': ['rbf'], 
                  'gamma': [ 0.001, 0.01, 0.1, 1],
                  'C': [1, 10, 50, 100,200,300, 1000]}

gsSVMC = GridSearchCV(SVMC,param_grid = svc_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsSVMC.fit(X_train,Y_train)

SVMC_best = gsSVMC.best_estimator_

# Best score
gsSVMC.best_score_

## 6.1.3 Plot learning curve
- 러닝커브를 파악하는 방법은 오버피팅을 파악하기 위한 좋은 방법입니다.

In [71]:

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    """Generate a simple plot of the test and training learning curve"""
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

g = plot_learning_curve(gsRFC.best_estimator_,"RF mearning curves",X_train,Y_train,cv=kfold)
g = plot_learning_curve(gsExtC.best_estimator_,"ExtraTrees learning curves",X_train,Y_train,cv=kfold)
g = plot_learning_curve(gsSVMC.best_estimator_,"SVC learning curves",X_train,Y_train,cv=kfold)
g = plot_learning_curve(gsadaDTC.best_estimator_,"AdaBoost learning curves",X_train,Y_train,cv=kfold)
g = plot_learning_curve(gsGBC.best_estimator_,"GradientBoosting learning curves",X_train,Y_train,cv=kfold)

- Gradient Boosting 과 Ada Classifier 은 훈련세트에 과대적합인 경향이 있습니다. 
- SVC 와 ExtraTrees는 일반적으로 좋은 경향이 있습니다. 왜냐하면 Training score 와 Cross_validation_score 인접하기 때문입니다.

## 6.1.4 Feature importance of tree based classifiers
- 승객의 생존에 대한 예측에 대해서 중요한 특징 정보를 얻기 위해서는 feature 의 중요도를 그래프로 시각화 해보겠습니다.

In [72]:
nrows = ncols = 2
fig, axes = plt.subplots(nrows = nrows, ncols = ncols, sharex = 'all', figsize = (15, 15))
names_classifiers = [("AdaBoosting", ada_best),("ExtraTrees",ExtC_best),("RandomForest",RFC_best),("GradientBoosting",GBC_best)]
nclassifier = 0
for row in range(nrows):
    for col in range(ncols):
        name = names_classifiers[nclassifier][0]
        classifier = names_classifiers[nclassifier][1]
        indices = np.argsort(classifier.feature_importances_)[::-1][:40]
        g = sns.barplot(y=X_train.columns[indices][:40],x = classifier.feature_importances_[indices][:40] , orient='h',ax=axes[row][col])
        g.set_xlabel("Relative importance",fontsize=12)
        g.set_ylabel("Features",fontsize=12)
        g.tick_params(labelsize=9)
        g.set_title(name + " feature importance")
        nclassifier += 1

- 4개의 분류기에서의 특성의 중요도가 모두 다르다는 것을 알 수 있습니다. 하지만 공통적으로 중요하다고 생각되는 Feature들은 'Fare', 'Title','Age','Sex' Feature 입니다.
- 의미
    - Pc_1, Pc_2, Pc_3 (Pclass)와 Fare은 승객의 일반적인 사회적 지위를 의미합니다.
    - Sex, Title_2 , Title_3 는 성별을 의미합니다.
    - Age, Title_1은 나이를 의미합니다. 
    - Fsize, LargeF, MedF, Single 은 가족의 인원을 의미힙니다.


#### 4가지 분류에 따르면, 승객의 자리보다는 승객의 연령, 성별, 가족의 규모, 사회적 지위가 더 관련있는 것을 의미합니다.

In [73]:

test_Survived_RFC = pd.Series(RFC_best.predict(test), name="RFC")
test_Survived_ExtC = pd.Series(ExtC_best.predict(test), name="ExtC")
test_Survived_SVMC = pd.Series(SVMC_best.predict(test), name="SVC")
test_Survived_AdaC = pd.Series(ada_best.predict(test), name="Ada")
test_Survived_GBC = pd.Series(GBC_best.predict(test), name="GBC")


# Concatenate all classifier results
ensemble_results = pd.concat([test_Survived_RFC,test_Survived_ExtC,test_Survived_AdaC,test_Survived_GBC, test_Survived_SVMC],axis=1)


g= sns.heatmap(ensemble_results.corr(),annot=True)

- 5개의 모델이 각자 차이가 존재하므로, 앙상블로 사용하기에 충분합니다.

## 6.2 Ensemble modeling

#### 6.2.1 모델 결합


In [74]:
votingC = VotingClassifier(estimators=[('rfc', RFC_best), ('extc', ExtC_best),
('svc', SVMC_best), ('adac',ada_best),('gbc',GBC_best)], voting='soft', n_jobs=4)

votingC = votingC.fit(X_train, Y_train)


In [90]:
test_Survived = pd.Series(votingC.predict(test), name="Survived")

results = pd.concat([IDtest,test_Survived],axis=1)

results.to_csv("submission.csv",index=False)

results